# Semantic Search TF-IDF Model First Draft

This Jupyter notebook is meant to serve as an introduction to reading Github `.md` documentation and analyzing it...

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tfidf_semantic_search as semsearch
import helper_funcs as helper

preproc_docs = helper.read_clean_process_data("docs/docs");
vectorizer, tfidf_matrix = helper.load_tfidf()

In [3]:
query = "How do I start a new workflow?"

semsearch.get_relevant_files(query, preproc_docs, vectorizer, tfidf_matrix, include_score=True, verbose=True);

Top 5 most relevant files to your query with similarity scores included:

1. Running Workflows: 0.2151150016737541
2. Restarting Your Workspace: 0.1701258712599974
3. Creating Clusters: 0.161842437326808
4. Navigating The Platform: 0.15312031446707564
5. Starting Stopping Clusters: 0.1185833979511654
